### NOUN Dataset + BLIP-2 Multimodal Model Pipeline
#### This notebook contains the pipeline for loading the BLIP2 Opt-2.7b model and running inference on the NOUN Dataset

Note that for this pipeline it is recommended to use a GPU with sufficient RAM.

##### Imports
Import modules, requires the installation of bitsandbytes and accelerate

In [ ]:
# %pip install bitsandbytes accelerate Pillow git+https://github.com/huggingface/transformers tqdm

In [3]:
%pip install bert-score

  Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
  Using cached matplotlib-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.40.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.2 MB)
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 25.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [45]:
import csv
from PIL import Image
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch
from tqdm import tqdm
from scipy.stats import pearsonr

import re
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import json

from PIL import Image
from IPython.display import HTML
from html_formatter import get_thumbnail, image_formatter
from evaluate import check_colors_and_textures, colors_to_boolean, textures_to_boolean

##### Load model
uses bitsandbytes to allow int8 quanitization for greatly reduced memory usage, allowing the model to be run on Google Colab.

In [2]:
# load processor
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")

# load in float16 # load in int8
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b",
                                                      load_in_8bit=True, device_map="auto")
# setup device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /gpfs/home5/jsprott/thesis-novel-objects/noun2-env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /sw/arch/RHEL8/EB_production/2022/software/CUDA/11.7.0/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /gpfs/home5/jsprott/thesis-novel-objects/noun2-env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

##### Perform inference on NOUN Dataset
Currently uses default hyperparameters

In [ ]:
import pandas as pd
from tqdm import tqdm
from evaluate import check_colors_and_textures

# Define path to input and output files
input_file = 'data/datasets/dataset_full.csv'
output_file = 'data/datasets/dataset_final_inference.csv'
extra_file = 'data/datasets/dataset_bonus_inference.csv'

# Define question templates
questions = [
    "Q: Only answer with yes or no, do you recognize this object? \n A:",
    "Q: what do you call the object in this image? \n A:",
    "Q: What do you really think this is? \n A:"
]

# Load data from input file into a pandas DataFrame
data = pd.read_csv(input_file)

tqdm.pandas()

# Define function to generate text using the model
def generate_text(row, decode='greedy', token_length=20, beams=5, top_p=0.9):
    raw_image = Image.open(row[0].replace("\\", "/")).convert("RGB")
    inputs = [processor(raw_image, return_tensors="pt").to(DEVICE, torch.float16)] + [processor(raw_image, text=q, return_tensors="pt").to(DEVICE, torch.float16) for q in questions]

    if decode == 'greedy':
        generated_ids = [model.generate(**input_, max_new_tokens=token_length) for input_ in inputs]
    elif decode == 'nucleus':
        generated_ids = [model.generate(**input_, do_sample=True, top_p=top_p, max_new_tokens=token_length) for input_ in inputs]
    elif decode == 'beam':
        generated_ids = [model.generate(**input_, num_beams=beams, max_new_tokens=token_length, repetition_penalty=1.0, length_penalty=1.0, temperature=1) for input_ in inputs]

    generated_texts = [processor.batch_decode(generated_id, skip_special_tokens=True)[0].strip() for generated_id in generated_ids]
    matches = [check_colors_and_textures(text) for text in generated_texts]

    return generated_texts + matches

# Apply the generate_text function to each row in the DataFrame
decode_methods = ['greedy', 'nucleus', 'beam']
columns = ['caption', 'bool', 'name', 'real', 'caption saliency', 'bool saliency', 'color saliency', 'real saliency']

for method in decode_methods:
    generated_cols = [f'BLIP-2, {method}, {column}' for column in columns]
    generated_data = data.progress_apply(lambda row: generate_text(row, decode=method), axis=1)
    data[generated_cols] = pd.DataFrame(np.array(generated_data).T.tolist(), columns=generated_cols)


display(data)

# Write updated data to output file
data.to_csv(output_file, index=False)

#### Ablative study: token lenght for greedy

In [97]:
method = 'greedy'
params = [5, 10, 20]
columns = ['caption', 'bool', 'name', 'real', 'caption saliency', 'bool saliency', 'color saliency', 'real saliency']

for param in params:
    generated_cols = [f'BLIP-2, {method}, max_new_tokens={param}, {column}' for column in columns]
    generated_data = data.progress_apply(lambda row: generate_text(row, decode=method, token_length=param), axis=1)
    data[generated_cols] = pd.DataFrame(np.array(generated_data).T.tolist(), columns=generated_cols)

100%|██████████| 64/64 [05:20<00:00,  5.01s/it]


#### Ablative study: top_p for nucleus sampling

In [98]:
method = 'nucleus'
params = [0.1, 0.5, 0.9]
columns = ['caption', 'bool', 'name', 'real', 'caption saliency', 'bool saliency', 'color saliency', 'real saliency']

for param in params:
    generated_cols = [f'BLIP-2, {method}, top_p={param}, {column}' for column in columns]
    generated_data = data.progress_apply(lambda row: generate_text(row, decode=method, top_p=param), axis=1)
    data[generated_cols] = pd.DataFrame(np.array(generated_data).T.tolist(), columns=generated_cols)

100%|██████████| 64/64 [06:32<00:00,  6.14s/it]


#### Ablative study: num_beams for beam search

In [100]:
method = 'beam'
params = [1, 5, 10]
columns = ['caption', 'bool', 'name', 'real', 'caption saliency', 'bool saliency', 'color saliency', 'real saliency']

for param in params:
    generated_cols = [f'BLIP-2, {method}, num_beams={param}, {column}' for column in columns]
    generated_data = data.progress_apply(lambda row: generate_text(row, decode=method, beams=param), axis=1)
    data[generated_cols] = pd.DataFrame(np.array(generated_data).T.tolist(), columns=generated_cols)

100%|██████████| 64/64 [11:39<00:00, 10.93s/it]


In [104]:
data.to_csv(extra_file, index=False)

#### Display data

##### Reformat color extraction and add image objects for html display

In [2]:
data = pd.read_csv('data/datasets/dataset_bonus_inference.csv')

In [3]:
columns = list(data.columns)
columns.remove('color saliency')
columns.remove('texture saliency')
string_to_match = 'saliency'

for column in columns:
    if string_to_match in column:
        data[column] = data[column].apply(lambda x: re.sub(r'[^\w]', ' ', x))

        column_color = re.sub(r'\s+saliency', ' to boolean', re.sub(r'BLIP-2', 'color', column))
        column_texture = re.sub(r'\s+saliency', ' to boolean', re.sub(r'BLIP-2', 'texture', column))

        data[column_color] = data[column].apply(lambda x: colors_to_boolean(x))
        data[column_texture] = data[column].apply(lambda x: textures_to_boolean(x))


In [ ]:
display(data)

In [51]:
#data.rename(columns={'image_path': 'image'}, inplace=True)
#data['image'] = data.image.map(lambda f: get_thumbnail(f))

/scratch-local/jsprott.2829221/ipykernel_4099205/257187248.py:13: DeprecationWarning:

LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.



Export data

In [71]:
html_content = data.to_html(formatters={'image': image_formatter}, escape=False)

with open('data/datasets/sorted_results.html', 'w') as file:
    file.write(html_content)
    
output_file = 'data/datasets/dataset_with_bert.csv'
data.to_csv(output_file, index=False)

## Evaluation

Check whether model uses color or texture terms

In [ ]:
get_token_count = lambda text: len(text.split())

data['length greedy'] = data['BLIP-2, greedy, caption'].apply(get_token_count)
data['length nucleus'] = data['BLIP-2, nucleus, caption'].apply(get_token_count)
data['length beam'] = data['BLIP-2, beam, caption'].apply(get_token_count)

In [26]:
data.columns

Index(['image_path', 'number label', 'actual name', 'familiarity score',
       'nameability score', 'color saliency', 'texture saliency',
       'BLIP-2, greedy, caption', 'BLIP-2, greedy, bool',
       'BLIP-2, greedy, name',
       ...
       'color, beam, num_beams=5, real to boolean',
       'texture, beam, num_beams=5, real to boolean',
       'color, beam, num_beams=10, caption to boolean',
       'texture, beam, num_beams=10, caption to boolean',
       'color, beam, num_beams=10, bool to boolean',
       'texture, beam, num_beams=10, bool to boolean',
       'color, beam, num_beams=10, name to boolean',
       'texture, beam, num_beams=10, name to boolean',
       'color, beam, num_beams=10, real to boolean',
       'texture, beam, num_beams=10, real to boolean'],
      dtype='object', length=199)

In [25]:
fig = px.scatter(data, size='color saliency', color='texture saliency', y='nameability score', x='familiarity score')
fig.update_layout(
    width=1000,  # Adjust the width of the plot as desired
    height=1000,  # Adjust the height of the plot as desired
)
fig.show()

In [28]:
import visualizations
import importlib
importlib.reload(visualizations)
from visualizations import visualize_saliency

figs = visualize_saliency(data, 'texture')

for fig in figs:
    fig.show()

In [60]:
import plotly.graph_objects as go
import pandas as pd

# Assuming your dataframe is called 'df'
# You can load your data into a dataframe using pd.read_csv() or any other method

# Create a list of row indices to use as labels for x-axis
labels = data['number label'].tolist()

# Create a figure object
fig = go.Figure()

# Add the bars for each length column
fig.add_trace(go.Bar(x=labels, y=data['length greedy'], name='Length Greedy'))
fig.add_trace(go.Bar(x=labels, y=data['length nucleus'], name='Length Nucleus'))
fig.add_trace(go.Bar(x=labels, y=data['length beam'], name='Length Beam'))

# Update the layout
fig.update_layout(
    width=1000,
    title='Grouped Bar Chart of token lengths',
    xaxis_title='Object label',
    yaxis_title='Token length',
    barmode='group',  # Set the bar mode to 'group' for grouped bars
    bargap=0.1,  # Adjust the spacing between bars within each group
    bargroupgap=0.3  # Adjust the spacing between groups
)

# Show the plot
fig.show()

# # Save the plot as JSON
fig_json = fig.to_json()
with open('token_length.json', 'w') as file:
    file.write(fig_json)

#### Bool breakdown

In [44]:
num_bins = 5
category_bins_familiarity = pd.cut(data['familiarity score'], bins=num_bins)

data_plot = data.copy()

data_plot['category bin familiarity'] = category_bins_familiarity.astype(str)

data_plot['greedy bool count'] = data_plot["BLIP-2, greedy, bool"].str.lower().str.count(r'\byes\b')
data_plot['nucleus bool count'] = data_plot["BLIP-2, nucleus, bool"].str.lower().str.count(r'\byes\b')
data_plot['beam bool count'] = data_plot["BLIP-2, beam, bool"].str.lower().str.count(r'\byes\b')

f_beam = data_plot.groupby('category bin familiarity')[
    'beam bool count'].mean()
f_greedy = data_plot.groupby('category bin familiarity')[
    'greedy bool count'].mean()
f_nucleus = data_plot.groupby('category bin familiarity')[
    'nucleus bool count'].mean()


fig = go.Figure()
# data=[go.Bar(x=bin_percentages.index.astype(str), y=bin_percentages)])

fig.add_trace(go.Bar(x=f_greedy.index.astype(
    str), y=f_greedy, name='Greedy decoding'))
fig.add_trace(go.Bar(x=f_nucleus.index.astype(
    str), y=f_nucleus, name='Nucleus Sampling'))
fig.add_trace(go.Bar(x=f_beam.index.astype(str), y=f_beam, name='Beam Search'))

fig.update_layout(width=800, xaxis_title='Bins', yaxis_title='Rate of model answering "yes"', xaxis=dict(title='Familiarity score in %', tickmode='array',
                                                                                                             tickvals=[
                                                                                                                 0, 1, 2, 3, 4],
                                                                                                             ticktext=['0-20', '21-40', '41-60', '61-80', '81-100']))
fig.show()

# fig.show()

# # Save the plot as JSON
# fig_json = fig.to_json()
# with open('familiarity.json', 'w') as file:
#     file.write(fig_json)

In [61]:
data.columns

Index(['image_path', 'number label', 'actual name', 'familiarity score',
       'nameability score', 'color saliency', 'texture saliency',
       'BLIP-2, greedy, caption', 'BLIP-2, greedy, bool',
       'BLIP-2, greedy, name',
       ...
       'color, beam, num_beams=10, caption to boolean',
       'texture, beam, num_beams=10, caption to boolean',
       'color, beam, num_beams=10, bool to boolean',
       'texture, beam, num_beams=10, bool to boolean',
       'color, beam, num_beams=10, name to boolean',
       'texture, beam, num_beams=10, name to boolean',
       'color, beam, num_beams=10, real to boolean',
       'texture, beam, num_beams=10, real to boolean', 'category bin color',
       'category bin texture'],
      dtype='object', length=201)

In [60]:
for c in data.columns:
    if 'color' in c:
        print(c)

color saliency
color, greedy, caption to boolean
color, greedy, bool to boolean
color, greedy, name to boolean
color, greedy, real to boolean
color, nucleus, caption to boolean
color, nucleus, bool to boolean
color, nucleus, name to boolean
color, nucleus, real to boolean
color, beam, caption to boolean
color, beam, bool to boolean
color, beam, name to boolean
color, beam, real to boolean
color, greedy, max_new_tokens=5, caption to boolean
color, greedy, max_new_tokens=5, bool to boolean
color, greedy, max_new_tokens=5, name to boolean
color, greedy, max_new_tokens=5, real to boolean
color, greedy, max_new_tokens=10, caption to boolean
color, greedy, max_new_tokens=10, bool to boolean
color, greedy, max_new_tokens=10, name to boolean
color, greedy, max_new_tokens=10, real to boolean
color, greedy, max_new_tokens=20, caption to boolean
color, greedy, max_new_tokens=20, bool to boolean
color, greedy, max_new_tokens=20, name to boolean
color, greedy, max_new_tokens=20, real to boolean
col

In [67]:
corr, p_value = pearsonr(list(data['color saliency']), list(data['color, beam, num_beams=10, caption to boolean']))

data_to_corr = ['color saliency', 'texture saliency', 'nameability score', 'familiarity score']
correlations = {}

for column in data_to_corr:
    for item in data_plot.columns:
        if column == 'color saliency' and column != item and 'bin' not in item:
            if 'color' in item:
                correlations[item] = pearsonr(list(data['color saliency']), list(data[item]))[0]
        if column == 'texture saliency' and column != item and 'bin' not in item:
            if 'texture' in item:
                correlations[item] = pearsonr(list(data['texture saliency']), list(data[item]))[0]
        if column == 'familiarity score' and column != item and 'bin' not in item:
            if 'bool count' in item:
                correlations[item] = pearsonr(list(data_plot['familiarity score']), list(data_plot[item]))[0]


In [81]:
correlations.keys()

dict_keys(['color, greedy, caption to boolean', 'color, greedy, bool to boolean', 'color, greedy, name to boolean', 'color, greedy, real to boolean', 'color, nucleus, caption to boolean', 'color, nucleus, bool to boolean', 'color, nucleus, name to boolean', 'color, nucleus, real to boolean', 'color, beam, caption to boolean', 'color, beam, bool to boolean', 'color, beam, name to boolean', 'color, beam, real to boolean', 'color, greedy, max_new_tokens=5, caption to boolean', 'color, greedy, max_new_tokens=5, bool to boolean', 'color, greedy, max_new_tokens=5, name to boolean', 'color, greedy, max_new_tokens=5, real to boolean', 'color, greedy, max_new_tokens=10, caption to boolean', 'color, greedy, max_new_tokens=10, bool to boolean', 'color, greedy, max_new_tokens=10, name to boolean', 'color, greedy, max_new_tokens=10, real to boolean', 'color, greedy, max_new_tokens=20, caption to boolean', 'color, greedy, max_new_tokens=20, bool to boolean', 'color, greedy, max_new_tokens=20, name t

### BERTScore 


In [84]:
import bert_score
from bert_score import score

# hide the loading messages
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams["xtick.major.size"] = 0
rcParams["xtick.minor.size"] = 0
rcParams["ytick.major.size"] = 0
rcParams["ytick.minor.size"] = 0

rcParams["axes.labelsize"] = "large"
rcParams["axes.axisbelow"] = True
rcParams["axes.grid"] = True

#### Get candidate and reference sentences

In [86]:
refs = list(data['actual name'])
candidates = {}
title = 'BLIP-2'
columns = data.columns
for column in columns:
    if title in column and 'saliency' not in column and 'bool' not in column:
        candidates[column] = list(data[column])


# cands_greedy_caption = list(data['BLIP-2, greedy, caption'])
# cands_beam_caption = list(data['BLIP-2, beam, caption'])
# cands_nucleus_caption = list(data['BLIP-2, nucleus, caption'])


# cands_greedy_name = data['BLIP-2, greedy, name']
# cands_beam_name = data['BLIP-2, beam, name']
# cands_nucleus_name = data['BLIP-2, nucleus, name']

# cands_greedy_real = list(data['BLIP-2, greedy, real'])
# cands_beam_real = list(data['BLIP-2, beam, real'])
# cands_nucleus_real = list(data['BLIP-2, nucleus, real'])

#### BERTScore familiarity measurement
BERTScore produces P, R, F1 scores respectively. 

In [ ]:
familiarity_scores = {}
for item in candidates.items():
    familiarity_scores[f'{item[0]} Familiarity BERTScore'] = score(item[1], refs, lang='en', verbose=True)

# data2['BERTScore greedy real'] = score(cands_greedy_real, refs, lang='en', verbose=True)[2]
# data2['BERTScore beam real'] = score(cands_beam_real, refs, lang='en', verbose=True)[2]
# data2['BERTScore nucleus real'] = score(cands_nucleus_real, refs, lang='en', verbose=True)[2]

In [39]:
familiarity_scores['BLIP-2, greedy, caption Familiarity BERTScore']

(tensor([0.8100, 0.8221, 0.8494, 0.7979, 0.8287, 0.8014, 0.8527, 0.8148, 0.8464,
         0.8302, 0.7824, 0.8600, 0.8061, 0.8453, 0.8267, 0.8207, 0.8146, 0.8293,
         0.8184, 0.8167, 0.8032, 0.7891, 0.8105, 0.8344, 0.8095, 0.8137, 0.8245,
         0.8056, 0.8232, 0.7827, 0.8170, 0.7771, 0.8515, 0.8520, 0.8550, 0.8462,
         0.8303, 0.8593, 0.8085, 0.8299, 0.8106, 0.8203, 0.8099, 0.8786, 0.8380,
         0.8448, 0.7913, 0.8571, 0.8275, 0.8759, 0.8097, 0.7968, 0.7645, 0.7945,
         0.8423, 0.8241, 0.8146, 0.8476, 0.8099, 0.8781, 0.8192, 0.8389, 0.7995,
         0.7825]),
 tensor([0.8491, 0.8048, 0.8636, 0.8217, 0.8772, 0.8163, 0.8098, 0.8284, 0.9657,
         0.9565, 0.7715, 0.8292, 0.8273, 0.8872, 0.8816, 0.9148, 0.7923, 0.8604,
         0.8300, 0.8138, 0.8911, 0.8727, 0.8718, 0.8835, 0.7847, 0.9488, 0.8380,
         0.8737, 0.8370, 0.8701, 0.8005, 0.8592, 0.9018, 0.8451, 0.8374, 0.9141,
         0.7785, 0.8808, 0.7834, 0.9186, 0.8079, 0.8587, 0.7844, 0.9166, 0.8702,
         

BERTScore nameability measurement

In [93]:
# data2['BERTScore nameability caption'] = score(cands_greedy_caption, cands_nucleus_caption, lang='en', verbose=True)[2] + score(
#     cands_greedy_caption, cands_beam_caption, lang='en', verbose=True)[2] + score(cands_beam_caption, cands_nucleus_caption, lang='en', verbose=True)[2]



keys = list(candidates.keys())
n = len(keys)

mean = 0
precisions = np.zeros(64)
recalls = np.zeros(64)
f1_scores = np.zeros(64)

for i in range(n - 1):
    for j in range(i + 1, n):
        key1 = keys[i]
        key2 = keys[j]
        if candidates[key1] != candidates[key2]:  # Exclude pairs with two identical lists
            if 'caption' in key1 and 'caption' in key2:
                p, r, f1 = score(candidates[key1], candidates[key2], lang='en', verbose=True)
                precisions += np.array(p)
                recalls += np.array(r)
                f1_scores += np.array(f1)
                mean += 1
                print(f1_scores)
print(f1_scores/mean)


calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00,  6.62it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 66.25it/s]


done in 0.32 seconds, 199.37 sentences/sec
[0.90649217 0.95129001 0.85559881 0.88445354 0.92752224 0.7967003
 0.94441569 0.84227252 0.91361856 0.85112768 0.85414207 0.92016542
 0.9288013  0.93958044 0.9401778  0.82912558 0.86086822 0.90401286
 0.89034086 0.91342008 0.890715   0.87854522 0.92492259 0.84161073
 0.90977031 0.86307645 0.93932515 0.8927896  0.87861329 0.9061051
 0.88382542 0.8324846  0.89714229 0.91354966 0.83259296 0.95261687
 0.88848847 0.91238475 0.89543301 0.90840459 0.85437393 0.94840646
 0.86646992 0.90354478 0.86432469 0.89571035 0.86466801 0.92304814
 0.92141885 0.94400597 0.85966688 0.9576112  0.9185617  0.93780416
 0.91403228 0.84792918 0.94170374 0.93093139 0.8984921  0.91508621
 0.88439023 0.98458844 0.81471246 0.90929568]
calculating scores...
computing bert embedding.


...

done in 0.31 seconds, 205.46 sentences/sec
[56.71632266 56.96608311 54.95373189 57.03578568 57.27848935 54.23500806
 57.7496199  55.39068836 56.47017241 56.25680578 56.31445724 55.97181964
 56.56917799 56.81018287 57.87597805 55.70034647 56.90661603 57.12851697
 57.82361114 55.9909153  55.1490742  55.90695864 55.74337268 54.08173567
 56.13144946 56.67189902 56.60211843 56.10698521 54.93832928 56.81022471
 56.19568968 55.29425967 58.00754672 57.96114987 55.70100349 57.15812391
 56.82393336 55.08110309 56.91832572 56.80617374 56.25774157 57.57677895
 56.24583119 55.63564187 57.4358989  56.26287949 55.86101604 57.79660124
 56.3576293  57.41226304 57.87803888 58.62147659 55.72841424 57.33931673
 57.18956584 55.88352072 58.48623854 58.21575278 57.52725792 56.13927501
 56.87355745 57.25598991 56.22949147 55.64114261]
calculating scores...
computing bert embedding.


100%|██████████| 2/2 [00:00<00:00, 10.00it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 68.92it/s]

done in 0.22 seconds, 292.49 sentences/sec
[57.63561893 57.96608299 55.95373178 58.03578591 58.27848959 55.176157
 58.7496199  56.35031569 57.47017235 57.25680578 57.2532106  56.97181976
 57.49508142 57.81018263 58.87597781 56.70034659 57.90661603 58.12851697
 58.82361114 56.99091542 56.0752877  56.90695858 56.68859416 55.04195207
 57.03287727 57.66487908 57.57204378 57.10698521 55.93832916 57.81022483
 57.06958991 56.29425967 58.99044937 58.96115011 56.6620872  58.15812379
 57.72673357 55.93582034 57.91832566 57.78409576 57.21767294 58.57677895
 57.24583113 56.54927832 58.4358989  57.26287973 56.86101627 58.79660112
 57.35762918 58.37478727 58.87803888 59.62147671 56.72841412 58.33931673
 58.18956578 56.88352072 59.48623854 59.21575278 58.52725792 57.07904512
 57.87355733 58.25598985 57.20263833 56.64114261]
[0.92960676 0.93493682 0.90247954 0.93606106 0.93997564 0.88993802
 0.94757451 0.90887606 0.92693826 0.92349687 0.92343888 0.91890032
 0.92734002 0.9324223  0.94961255 0.91452172 

In [92]:
print(precisions)

[1.88258737 1.9012568  1.807046   1.82560503 1.87695694 1.7411176
 1.94123811 1.75006062 1.8324486  1.78149503 1.83272487 1.84700441
 1.86065108 1.86720437 1.84578151 1.77506191 1.82262945 1.90253288
 1.89925003 1.87469304 1.77048856 1.79573447 1.84688228 1.7519778
 1.8544668  1.86708915 1.8549484  1.79748917 1.79987949 1.84800816
 1.84801626 1.77539831 1.86010724 1.91572446 1.73276013 1.94701809
 1.88019603 1.82258058 1.84593779 1.87070096 1.81863201 1.9129253
 1.83640128 1.82859474 1.81312287 1.84606659 1.80880338 1.92118859
 1.82205242 1.90572929 1.86698109 1.89267331 1.81808591 1.91042399
 1.91256136 1.84877467 1.93013561 1.94453168 1.88956279 1.84025955
 1.84130287 1.88679069 1.80555153 1.78186572]


BERTScore Visualization

In [4]:
import plotly.graph_objects as go
import pandas as pd

# Assuming your dataframe is called 'data'
# You can load your data into a dataframe using pd.read_csv() or any other method

# Sort the data based on the column with values between 0 and 100
sorted_data = data.sort_values('nameability score')

# Create a list of row indices to use as labels for x-axis
labels = sorted_data['number label'].tolist()

# Create a figure object
fig = go.Figure()

# Add the bars for each length column
fig.add_trace(go.Bar(x=labels, y=sorted_data['BERTScore greedy real'], name='Greedy real object question'))
fig.add_trace(go.Bar(x=labels, y=sorted_data['BERTScore nucleus real'], name='Nucleus image captioning'))
fig.add_trace(go.Bar(x=labels, y=sorted_data['BERTScore beam real'], name='Beam real object question'))

# Update the layout
fig.update_layout(
    width=1000,
    title='Bar Chart of BERT Scores: nameability',
    xaxis_title='Nameability human score',
    yaxis_title='BERT Score',
    yaxis_type='log',
    bargap=0.1,  # Adjust the spacing between bars within each group
    bargroupgap=0.3  # Adjust the spacing between groups
)

# Show the plot
fig.show()

# # # Save the plot as JSON
# fig_json = fig.to_json()
# with open('token_length.json', 'w') as file:
#     file.write(fig_json)


NameError: name 'data' is not defined